# Unsupervised Learning
## DSSP Team
## Summer 2020

## Clustering

We are going to work on a synthetic dataset that examplifies several type of _clusters_.

We need first to read the data.

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import fastcluster
sns.set_style('whitegrid')

from sklearn import cluster
from sklearn import metrics
from scipy.spatial.distance import squareform
from scipy.cluster import hierarchy

In [ ]:
don = pd.read_csv('../data/donclassif.txt.gz', sep=';')

To visualize it, we will use the following function:

In [ ]:
def plot_don(labels=None, s=10):
    if labels is None:
        labels = np.random.permutation(len(don.index))
    plt.scatter(x=don['V1'], y=don['V2'], c=labels, s=s)
    plt.axis('equal')

In [ ]:
plot_don()

__1)__ What kind of _clusters_ can you observe?


In [ ]:
#solution
# We observe clusters characterized by a center and a dispersion around it and 
# clusters corresponding to densily connected zones. 


__2)__ Perform a _k-means_ clustering for this dataset with $k=10$.

__Hint:__ Use the `cluster.KMeans` class from scikit-learn. 

In [ ]:
kmeans = cluster.KMeans(n_clusters=10).fit(don)
plot_don(kmeans.labels_)

__3)__ Perform several time the k-means clustering for the same value of $k$. 

__Hint:__ Use `plt.subplot` to wrap the plots

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(3):
    for j in range(3):
        plt.subplot(3, 3, 3 * i + j + 1)
        kmeans = cluster.KMeans(n_clusters=10).fit(don)
        plot_don(kmeans.labels_, s=3)

__4)__ Why are the results different?

In [ ]:
# solution
# There is a random initialization and the algorithm converges only to a local optimum.

__5)__ Try several values for $k$ for instance for

In [ ]:
k = np.array([2, 3, 4, 5, 10, 15, 20, 24, 40])

In [ ]:
#solution
plt.figure(figsize=(12, 12))
for i in range(3):
    for j in range(3):
        plt.subplot(3, 3, 3 * i + j + 1)
        kmeans = cluster.KMeans(n_clusters=k[3 * i + j]).fit(don)
        plot_don(kmeans.labels_, s=3)

__6)__ Use the `cluser.DBSCAN` function class from scikit-learn with `eps=.2` to cluster the data.


In [ ]:
# solution 
dbscan = cluster.DBSCAN(eps=.2).fit(don)
plot_don(dbscan.labels_)

__6)__ Study the effect of the `eps` parameter using for instance the following values:

In [ ]:
eps = np.logspace(-1, 0, num=9)  # logarithmically/geometrically spaced values
eps

In [ ]:
# solution
plt.figure(figsize=(12, 12))
for i in range(3):
    for j in range(3):
        plt.subplot(3, 3, 3 * i + j + 1)
        dbscan = cluster.DBSCAN(eps=eps[j+3*i]).fit(don)
        plot_don(dbscan.labels_, s=3)

__8)__ The last type of clustering methods we consider is the hierarchical one. The best implementation is available in the `fastcluster` package. It is optimized for memory usage when the `linkage_vector` function

In [ ]:
Z_single_vec = fastcluster.linkage_vector(don, method='single', metric='euclidean')

which is restricted to the most classical metric and methods and for speed with an arbitrary distance metric and the most classical methods when using the `linkage` function

In [ ]:
D = metrics.pairwise_distances(don)
Dcond = squareform(D, checks=False)  # more compact representation of the distance matrix
Z_single = fastcluster.linkage(Dcond, method='single')

Both yield the same clustering when used with the same parameters

In [ ]:
pd.DataFrame(data=Z_single_vec,
    columns=['clusterOne','clusterTwo','distance','newClusterSize'])

In [ ]:
pd.DataFrame(data=Z_single,
    columns=['clusterOne','clusterTwo','distance','newClusterSize'])

The underlying matrix encodes the hierarchy in a format compatible with the `scipy.cluster` format. We can thus display the cluster hierarchy (dendrogram, from greek *dendros*, tree) with:

In [ ]:
hierarchy.dendrogram(Z_single);

One has also access to the _flattening_ function `hierarchy.fcluster` that slices the hierarchy to give a label to each sample:

In [ ]:
labels_ = hierarchy.fcluster(Z_single, t=10, criterion='maxclust')

plot_don(labels_)
print(len(np.unique(labels_)))  # there are indeed 10 clusters returned

What happens if you slice the hierarchy with the different $k$ we have used before?

__9)__ Repeat the experiment with the following methods: 
`'complete'`, `'average'` and `'ward.D2'`.

In [ ]:
# solution
plt.figure(figsize=(12, 12))
for i in range(3):
    for j in range(3):
        plt.subplot(3, 3, 3 * i + j + 1)
        labels_ = hierarchy.fcluster(Z_single, t=k[3 * i + j], criterion='maxclust')
        plot_don(labels_, s=3)

In [ ]:
plt.figure(figsize=(12, 12))
Z_complete = fastcluster.linkage(Dcond, method='complete')
for i in range(3):
    for j in range(3):
        plt.subplot(3, 3, 3 * i + j + 1)
        labels_ = hierarchy.fcluster(Z_complete, t=k[3 * i + j], criterion='maxclust')
        plot_don(labels_, s=3)

In [ ]:
plt.figure(figsize=(12, 12))

Z_average = fastcluster.linkage(Dcond, method='average')
for i in range(3):
    for j in range(3):
        plt.subplot(3, 3, 3 * i + j + 1)
        labels_ = hierarchy.fcluster(Z_average, t=k[3 * i + j], criterion='maxclust')
        plot_don(labels_, s=3)

In [ ]:
plt.figure(figsize=(12, 12))

Z_ward = fastcluster.linkage(Dcond, method='ward')
for i in range(3):
    for j in range(3):
        plt.subplot(3, 3, 3 * i + j + 1)
        labels_ = hierarchy.fcluster(Z_ward, t=k[3 * i + j], criterion='maxclust')
        plot_don(labels_, s=3)

__10)__ What is your preferred clustering? Why?

__11)__ What happens if one uses the much larger dataset stored in `donclassif2.txt.gz`?